<a href="https://colab.research.google.com/github/shin0105/4YP/blob/master/Boston_housing_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing packages
!pip install jaxopt
!pip install tinygp

In [3]:
#importing packages
import numpy as np
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
import jaxopt
import tinygp
jax.config.update("jax_enable_x64", True)
import pandas as pd

In [ ]:
#reading csv file for Boston Housing Dataset
from google.colab import files
uploaded = files.upload()
import io
df2 = pd.read_csv(io.BytesIO(uploaded['hou_all.csv']))